In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
masoudnickparvar_brain_tumor_mri_dataset_path = kagglehub.dataset_download('masoudnickparvar/brain-tumor-mri-dataset')

print('Data source import complete.')


Data source import complete.


**IMPORTING LIBRARIES AND DEFINING DIRECTORIES**

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import VGG16, ResNet50
from sklearn.preprocessing import label_binarize
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import VGG16, ResNet50

In [3]:
train_dir = '/kaggle/input/brain-tumor-mri-dataset/Training'
test_dir = '/kaggle/input/brain-tumor-mri-dataset/Testing'

In [4]:
IMG_SIZE = (150, 150)
BATCH_SIZE = 32
EPOCHS = 50

**Data Augmentation**

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   zoom_range=0.2,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   horizontal_flip=True,
                                   validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=IMG_SIZE,
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='categorical',
                                                    subset='training',
                                                    shuffle=True)


validation_generator = train_datagen.flow_from_directory(train_dir,
                                                         target_size=IMG_SIZE,
                                                         batch_size=BATCH_SIZE,
                                                         class_mode='categorical',
                                                         subset='validation',
                                                         shuffle=False)


test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=IMG_SIZE,
                                                  batch_size=BATCH_SIZE,
                                                  class_mode='categorical',
                                                  shuffle=False)

In [ ]:
class_labels = list(train_generator.class_indices.keys())
num_classes = len(class_labels)
samples_per_class = 3
print(num_classes)
print(class_labels)

In [ ]:
plt.figure(figsize=(samples_per_class * 3, num_classes * 3))  # Create a large canvas

for idx, class_label in enumerate(class_labels):  # Loop over each class name

    # Find the corresponding class index (e.g., 'dog' → 1)
    class_index = train_generator.class_indices[class_label]

    # Get the full list of file paths and class indices from the generator
    filepaths = train_generator.filepaths
    classes = train_generator.classes

    # Filter the indices that belong to the current class
    indices = [i for i, c in enumerate(classes) if c == class_index]

    # Select the first few examples (e.g., 3 images) of this class
    selected = indices[:samples_per_class]

    # Loop through the selected image indices
    for j, i in enumerate(selected):
        img_path = filepaths[i]                                # Get the image path
        img = load_img(img_path, target_size=IMG_SIZE)         # Load and resize the image
        img_arr = img_to_array(img) / 255.0                    # Convert to array and normalize

        # Plot the image in a subplot
        ax = plt.subplot(num_classes, samples_per_class, idx * samples_per_class + j + 1)
        ax.imshow(img_arr.astype('float32'))                   # Show the image
        ax.set_title(f'{class_label}')                         # Show the class name
        ax.axis('off')                                         # Hide axis ticks

# Add a title above all subplots
plt.suptitle('Examples of Training Images per Class', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout so nothing overlaps
plt.show()


In [ ]:
class_labels = list(train_generator.class_indices.keys())
train_classes = train_generator.classes

plt.figure(figsize=(8, 6))
sns.countplot(x=[class_labels[c] for c in train_classes])
plt.title('Class Distribution in the Training Set')
plt.xlabel('Class')
plt.ylabel('Number of Images')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [14]:
# Compute class weights to balance training classes (optional)
class_weights_vals = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_classes),
    y=train_classes
)

class_weights_dict = {i: class_weights_vals[i] for i in range(len(class_weights_vals))}
print('Class weights:', class_weights_dict)


Class weights: {0: np.float64(1.0811258278145695), 1: np.float64(1.0659981343283582), 2: np.float64(0.8955721003134797), 3: np.float64(0.9800600343053173)}


**BUILDING CUSTOM CNN**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D, Conv2D, MaxPooling2D, Flatten

# Building a deeper CNN with Batch Normalization
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(class_labels), activation='softmax')  # Output layer for multi-class classification
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model architecture
model.summary()


In [ ]:
%%time

# Callbacks for early stopping and learning rate reduction
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, verbose=1)

# Train the model
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=[early_stop, lr_reduce]
)

**PLOTS AND TEST ACCURACY**

In [ ]:
# Plot training and validation curves (accuracy and loss)
fig, axs = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy curve
sns.lineplot(ax=axs[0], x=range(1, len(history.history['accuracy']) + 1), y=history.history['accuracy'], label='Training', color='blue')
sns.lineplot(ax=axs[0], x=range(1, len(history.history['val_accuracy']) + 1), y=history.history['val_accuracy'], label='Validation', color='red')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Accuracy')
axs[0].set_title('Training and Validation Accuracy')
axs[0].legend()

# Loss curve
sns.lineplot(ax=axs[1], x=range(1, len(history.history['loss']) + 1), y=history.history['loss'], label='Training Loss', color='orange')
sns.lineplot(ax=axs[1], x=range(1, len(history.history['val_loss']) + 1), y=history.history['val_loss'], label='Validation Loss', color='purple')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Loss')
axs[1].set_title('Training and Validation Loss')
axs[1].legend()

plt.tight_layout()
plt.show()


In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test loss: {test_loss:.4f}, Test accuracy: {test_acc:.4f}')


In [ ]:
# Predictions on the test set
predictions = model.predict(test_generator)

# Get the class with the highest probability for each sample
predicted_classes = np.argmax(predictions, axis=1)

# Get the true class labels from the test set
true_classes = test_generator.classes


In [ ]:
# Confusion Matrix
cm = confusion_matrix(true_classes, predicted_classes)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d',
            xticklabels=class_labels,
            yticklabels=class_labels,
            cmap='Blues')

plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# Classification report: shows precision, recall, f1-score for each class
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)


In [ ]:
# Visualizing some predictions using subplots
num_rows = 3
num_cols = 3
fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 12))
axes = axes.flatten()

# Ensure there are enough test images
total_test = len(test_generator.filenames)

for i, ax in enumerate(axes):
    if i >= total_test:
        break

    # Get a batch from the generator
    img_batch, label_batch = test_generator[i]

    # Get the first image and its labels
    img = img_batch[0]
    true_label = np.argmax(label_batch[0])  # Convert one-hot to class index
    pred_label = predicted_classes[i]       # Already computed from model.predict()

    ax.imshow(img)
    ax.set_title(f'True: {class_labels[true_label]}\nPred: {class_labels[pred_label]}')
    ax.axis('off')

plt.tight_layout()
plt.show()


** Model 2 – Using Pre-trained VGG16 or ResNet50**

In [ ]:
# Directories
train_dir = '/kaggle/input/brain-tumor-mri-dataset/Training'
test_dir = '/kaggle/input/brain-tumor-mri-dataset/Testing'
# Parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 20


In [ ]:
# Data augmentation for training, with 20% validation split
train_datagen = ImageDataGenerator(
    rescale=1./255,             # Normalize pixel values to [0, 1]
    rotation_range=20,          # Random rotation up to ±20 degrees
    zoom_range=0.2,             # Random zoom up to ±20%
    width_shift_range=0.1,      # Horizontal shift up to 10%
    height_shift_range=0.1,     # Vertical shift up to 10%
    horizontal_flip=True,       # Random horizontal flip
    validation_split=0.2        # Reserve 20% of training data for validation
)

# Only rescaling for test data (no augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)

# Training data generator (80% of train_dir)
train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',          # Use only the training part
    shuffle=True
)

# Validation data generator (20% of train_dir)
validation_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',        # Use only the validation part
    shuffle=False
)

# Test data generator (from separate test_dir)
test_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

class_labels = list(train_generator.class_indices.keys())
train_classes = train_generator.classes

plt.figure(figsize=(8, 6))
sns.countplot(x=[class_labels[c] for c in train_classes])
plt.title('Class Distribution in Training Set')
plt.xlabel('Class')
plt.ylabel('Number of Images')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Compute class weights to handle class imbalance
class_weights_vals = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_classes),
    y=train_classes
)
class_weights_dict = {i: class_weights_vals[i] for i in range(len(class_weights_vals))}
print('Class weights:', class_weights_dict)

In [ ]:
# Callbacks to stop early and reduce learning rate when validation loss plateaus
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, verbose=1)

# Function to build a transfer learning model
def build_model(base_name, input_shape, num_classes, freeze_layers=True):

    # Load the selected pretrained model without top layers
    if base_name == 'VGG16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    elif base_name == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    else:
        raise ValueError(f"Base model {base_name} not supported")

    # Optionally freeze base model layers
    if freeze_layers:
        for layer in base_model.layers:
            layer.trainable = False

    # Add custom classification head
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    # Build and compile the full model
    model = Model(inputs=base_model.input, outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model


In [ ]:
%%time

# Training and storing results
results = {}
input_shape = (IMG_SIZE[0], IMG_SIZE[1], 3)
num_classes = len(class_labels)

for base_name in ['VGG16', 'ResNet50']:
    print(f"\nTraining model with base {base_name}...")

    model = build_model(base_name, input_shape, num_classes, freeze_layers=True)

    history = model.fit(train_generator,
                        epochs=EPOCHS,
                        validation_data=validation_generator)

    # Evaluation on test set
    test_loss, test_acc = model.evaluate(test_generator)
    print(f'{base_name} - Test loss: {test_loss:.4f}, Test accuracy: {test_acc:.4f}')

    # Predictions on test set
    predictions = model.predict(test_generator)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_generator.classes

    # Store in results
    results[base_name] = {
        'model': model,
        'history': history,
        'predictions': predictions,
        'predicted_classes': predicted_classes,
        'true_classes': true_classes
    }


In [ ]:
# Plot training curves (accuracy and loss) for VGG16

history = results['VGG16']['history']  # Use VGG16 history; change key to plot other models

fig, axs = plt.subplots(1, 2, figsize=(14, 5))

# Plot accuracy
sns.lineplot(ax=axs[0], x=range(1, len(history.history['accuracy']) + 1),
             y=history.history['accuracy'], label='Training', color='blue')
sns.lineplot(ax=axs[0], x=range(1, len(history.history['val_accuracy']) + 1),
             y=history.history['val_accuracy'], label='Validation', color='red')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Accuracy')
axs[0].set_title('Accuracy - VGG16')
axs[0].legend()

# Plot loss
sns.lineplot(ax=axs[1], x=range(1, len(history.history['loss']) + 1),
             y=history.history['loss'], label='Training Loss', color='orange')
sns.lineplot(ax=axs[1], x=range(1, len(history.history['val_loss']) + 1),
             y=history.history['val_loss'], label='Validation Loss', color='purple')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Loss')
axs[1].set_title('Loss - VGG16')
axs[1].legend()

plt.tight_layout()
plt.show()


In [ ]:
# Plot training curves (accuracy and loss) for ResNet50

history = results['ResNet50']['history']  # Use ResNet50 history

fig, axs = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy plot
sns.lineplot(ax=axs[0], x=range(1, len(history.history['accuracy']) + 1),
             y=history.history['accuracy'], label='Training', color='blue')
sns.lineplot(ax=axs[0], x=range(1, len(history.history['val_accuracy']) + 1),
             y=history.history['val_accuracy'], label='Validation', color='red')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Accuracy')
axs[0].set_title('Accuracy - ResNet50')
axs[0].legend()

# Loss plot
sns.lineplot(ax=axs[1], x=range(1, len(history.history['loss']) + 1),
             y=history.history['loss'], label='Training Loss', color='orange')
sns.lineplot(ax=axs[1], x=range(1, len(history.history['val_loss']) + 1),
             y=history.history['val_loss'], label='Validation Loss', color='purple')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Loss')
axs[1].set_title('Loss - ResNet50')
axs[1].legend()

plt.tight_layout()
plt.show()


In [ ]:
# 1. Confusion Matrices for each model
for base_name, res in results.items():
    cm = confusion_matrix(res['true_classes'], res['predicted_classes'])
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d',
                xticklabels=class_labels,
                yticklabels=class_labels,
                cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Confusion Matrix - {base_name}')
    plt.show()


In [ ]:
# 2. Classification Report
for base_name, res in results.items():
    print(f'Classification report - {base_name}:')
    report = classification_report(res['true_classes'], res['predicted_classes'], target_names=class_labels)
    print(report)

In [ ]:
# 3. ROC Curves
for base_name, res in results.items():
    # Convert true labels to one-hot encoded format
    y_true_bin = label_binarize(res['true_classes'], classes=list(range(num_classes)))
    y_score = res['predictions']  # Model's predicted probabilities

    fpr = dict()  # False Positive Rates
    tpr = dict()  # True Positive Rates
    roc_auc = dict()  # Area Under Curve for each class

    # Compute FPR, TPR, and AUC for each class
    for i in range(num_classes):
        fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Plot ROC curves
    plt.figure(figsize=(8, 6))
    for i in range(num_classes):
        plt.plot(fpr[i], tpr[i], label=f'{class_labels[i]} (AUC = {roc_auc[i]:.2f})')

    # Add diagonal line for reference (random guessing)
    plt.plot([0, 1], [0, 1], 'k--', lw=1)

    # Format plot
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curves - {base_name}')
    plt.legend(loc='lower right')
    plt.show()


In [ ]:
# 4. Predictions in Subplots (Examples)
for base_name, res in results.items():
    print(f'Prediction examples - {base_name}')
    num_rows, num_cols = 3, 3
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 12))
    axes = axes.flatten()
    total_test = len(test_generator.filenames)

    for i, ax in enumerate(axes):
        if i >= total_test:
            break
        # Load the image for display
        img_path = test_generator.filepaths[i]
        img = load_img(img_path, target_size=IMG_SIZE)
        img_arr = img_to_array(img) / 255.0
        true_label = res['true_classes'][i]
        pred_label = res['predicted_classes'][i]

        ax.imshow(img_arr.astype('float32'))
        ax.set_title(f'True: {class_labels[true_label]}\nPred: {class_labels[pred_label]}')
        ax.axis('off')

    plt.suptitle(f'Test Set Predictions - {base_name}', fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()


In [ ]:
# 5. Comparison of Validation Accuracy between architectures
plt.figure(figsize=(8, 6))
for base_name, res in results.items():
    hist = res['history']
    sns.lineplot(
        x=range(1, len(hist.history['val_accuracy']) + 1),
        y=hist.history['val_accuracy'],
        label=f'Val Acc - {base_name}'
    )
plt.xlabel('Epoch')
plt.ylabel('Validation Accuracy')
plt.title('Validation Accuracy Comparison')
plt.legend()
plt.tight_layout()
plt.show()
